In [1]:
from __future__ import print_function
from itertools import count

import torch
import torch.autograd
import torch.nn.functional as F
from torch.autograd import Variable

POLY_DEGREE = 4
W_target = torch.randn(POLY_DEGREE, 1) * 5
b_target = torch.randn(1) * 5


def make_features(x):
    """Builds features i.e. a matrix with columns [x, x^2, x^3, x^4]."""
    x = x.unsqueeze(1)
    return torch.cat([x ** i for i in range(1, POLY_DEGREE+1)], 1)


def f(x):
    """Approximated function."""
    return x.mm(W_target) + b_target[0]


def poly_desc(W, b):
    """Creates a string description of a polynomial."""
    result = 'y = '
    for i, w in enumerate(W):
        result += '{:+.2f} x^{} '.format(w, len(W) - i)
    result += '{:+.2f}'.format(b[0])
    return result


def get_batch(batch_size=32):
    """Builds a batch i.e. (x, f(x)) pair."""
    random = torch.randn(batch_size)
    x = make_features(random)
    y = f(x)
    return Variable(x), Variable(y)


# Define model
fc = torch.nn.Linear(W_target.size(0), 1)

for batch_idx in count(1):
    # Get data
    batch_x, batch_y = get_batch()

    # Reset gradients
    fc.zero_grad()

    # Forward pass
    output = F.smooth_l1_loss(fc(batch_x), batch_y)
    loss = output.data[0]

    # Backward pass
    output.backward()

    # Apply gradients
    for param in fc.parameters():
        param.data.add_(-0.1 * param.grad.data)

    # Stop criterion
    if loss < 1e-3:
        break

print('Loss: {:.6f} after {} batches'.format(loss, batch_idx))
print('==> Learned function:\t' + poly_desc(fc.weight.data.view(-1), fc.bias.data))
print('==> Actual function:\t' + poly_desc(W_target.view(-1), b_target))

Loss: 0.000806 after 326 batches
==> Learned function:	y = -4.28 x^4 -3.99 x^3 -5.69 x^2 +7.97 x^1 -12.64
==> Actual function:	y = -4.25 x^4 -4.04 x^3 -5.70 x^2 +7.98 x^1 -12.62


In [2]:
fc

Linear (4 -> 1)

In [4]:
fc.weight.data.view?

In [5]:
W_target


-4.2462
-4.0365
-5.7014
 7.9812
[torch.FloatTensor of size 4x1]

In [10]:
POLY_DEGREE = 4
x = torch.randn(POLY_DEGREE, 1) * 5

In [11]:
x


 -4.1182
 -4.3893
 10.4702
  1.6236
[torch.FloatTensor of size 4x1]

In [12]:
torch.randn?

In [13]:
W_target = torch.randn(POLY_DEGREE, 1) * 5

In [16]:
W_target.size(0)

4

In [18]:
torch.randn(32).size()

torch.Size([32])

In [19]:
torch.randn(32).unsqueeze?

Object `unsqueeze` not found.


In [20]:
torch.randn(32).unsqueeze?

Object `unsqueeze` not found.


In [ ]:
torch.randn(32).unsqueeze

In [22]:
random = torch.randn(32)

In [29]:
random.unsqueeze(1)


-1.2143
-0.2562
-1.1258
 0.6102
 1.2501
-0.4114
-0.3935
-0.7745
 0.0098
-0.2912
 0.6179
-0.2813
-0.9924
-0.2944
 0.8345
-0.4749
 0.9456
-1.2041
-0.5864
-0.5924
-0.9683
-2.1012
 1.1020
 0.6738
 0.7678
 1.8439
 1.4827
-0.2586
-0.1395
 0.3851
 0.7537
 0.1688
[torch.FloatTensor of size 32x1]

In [30]:
torch.cat([random ** i for i in range(1, POLY_DEGREE+1)], 1)


-1.2143e+00  1.4744e+00 -1.7903e+00  2.1739e+00
-2.5624e-01  6.5658e-02 -1.6824e-02  4.3109e-03
-1.1258e+00  1.2674e+00 -1.4269e+00  1.6064e+00
 6.1024e-01  3.7240e-01  2.2725e-01  1.3868e-01
 1.2501e+00  1.5629e+00  1.9538e+00  2.4426e+00
-4.1140e-01  1.6925e-01 -6.9630e-02  2.8646e-02
-3.9352e-01  1.5486e-01 -6.0940e-02  2.3981e-02
-7.7450e-01  5.9985e-01 -4.6458e-01  3.5982e-01
 9.7608e-03  9.5272e-05  9.2993e-07  9.0768e-09
-2.9124e-01  8.4819e-02 -2.4702e-02  7.1943e-03
 6.1787e-01  3.8177e-01  2.3589e-01  1.4575e-01
-2.8133e-01  7.9145e-02 -2.2266e-02  6.2639e-03
-9.9237e-01  9.8480e-01 -9.7729e-01  9.6984e-01
-2.9442e-01  8.6683e-02 -2.5521e-02  7.5139e-03
 8.3452e-01  6.9642e-01  5.8117e-01  4.8500e-01
-4.7486e-01  2.2549e-01 -1.0708e-01  5.0848e-02
 9.4564e-01  8.9424e-01  8.4563e-01  7.9966e-01
-1.2041e+00  1.4498e+00 -1.7457e+00  2.1019e+00
-5.8638e-01  3.4384e-01 -2.0162e-01  1.1823e-01
-5.9239e-01  3.5093e-01 -2.0789e-01  1.2315e-01
-9.6831e-01  9.3763e-01 -9.0792e-01  8.

In [33]:
fc?

In [36]:
m = torch.nn.Linear(20, 30)
input = torch.autograd.Variable(torch.randn(128, 20))
output = m(input)
print(output.size())

torch.Size([128, 30])


In [37]:
toch.nn?

Object `toch.nn` not found.


In [39]:
torch.nn.Linear?

In [40]:
from torchvision import datasets

In [41]:
datasets.MNIST?